## Interest rate simulation code

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Concatenate, Dropout, Subtract, \
                        Flatten, MaxPooling2D, Multiply, Lambda, Add, Dot, RNN
from keras.backend import constant
from keras import optimizers
from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import Input
from keras import initializers
from keras.constraints import max_norm
from keras.utils import plot_model
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy

In [ ]:
# TIME GRID

N=12*20
time_grid_plain=np.linspace(0,N,N+1)
time_grid_dcc=time_grid_plain/12 # day-count-convention


In [ ]:
# YIELD CURVE PARAMETERS

index_yc=2135 # 31-Dec-2012

# YIELD CURVE DYNAMICS

# data import
path=''
ECB=pd.read_csv(path+'201231_ECB.csv')

# zero-coupon bond prices for maturity T in years and Svensson parameter z
def ZCB_prices(T,z):
    return np.exp(-1/100*(z[0]*T-(z[1]/z[4]+z[2]/z[4]**2)*(np.exp(-z[4]*T)-1)-
        z[2]/z[4]*T*np.exp(-z[4]*T)-z[3]/z[5]*T*np.exp(-z[5]*T)-z[3]/z[5]**2*
        (np.exp(-z[5]*T)-1)))

# historical discount factors for a time grid
vec_ZCB_prices=np.vectorize(ZCB_prices,excluded=[1])

# yields for historical day i
def hist_yc(i,grid=time_grid_dcc[1:],rating='AAA'):
    shift=0 if rating=='AAA' else 6
    z=[ECB.loc[i,:][1+shift],ECB.loc[i,:][2+shift],ECB.loc[i,:][3+shift]/\
        ECB.loc[i,:][5+shift],ECB.loc[i,:][4+shift]/ECB.loc[i,:][6+shift],1/\
        ECB.loc[i,:][5+shift],1/ECB.loc[i,:][6+shift]]
    return np.divide(-np.log(vec_ZCB_prices(grid,z)),grid)

# collection of historical yields
def vec_hist_yc(i,grid=time_grid_dcc[1:],rating='AAA'):
    output=hist_yc(i[0],grid,rating)
    for j in i[1:]:
        output=np.vstack((output,hist_yc(j,grid,rating)))
    return output

# transform yields into discount factors
def discounting(yields,terms):
    return np.exp(-yields*terms)

# PCA
hist_increments=vec_hist_yc(range(1,index_yc))-vec_hist_yc(range(0,index_yc-1))
PCA_mu=np.mean(hist_increments,axis=0)
PCA_Q=np.cov(hist_increments,rowvar=False)
PCA_lambda,PCA_Lambda=np.linalg.eig(PCA_Q)
PCA_lambda,PCA_Lambda=np.real(PCA_lambda),np.real(PCA_Lambda)
#print(np.cumsum(PCA_lambda)[:10]/np.sum(PCA_lambda))

# generate a new yield curve increment
def yc_scn_gen(n=3,PCA_mu=PCA_mu,PCA_Lambda=PCA_Lambda,PCA_lambda=PCA_lambda):
    return 22*PCA_mu+np.matmul(PCA_Lambda[:,:n],np.random.multivariate_normal(\
        np.zeros(n),np.diag(22*PCA_lambda[:n])).reshape(n,1)).flatten()
        



# YIELD CURVE OBJECTS

class yc:
    def __init__(self,yields):
        self.yields=yields
        self.dfs=discounting(yields,time_grid_dcc[1:])

    def cpn(self,tenor,maturity):
        return 12/tenor*(1-self.dfs[maturity-1])/np.sum(self.dfs[np.arange(tenor,maturity+tenor,tenor)-1])
 
        
    def pv(self,cash_flows):
        return np.dot(self.dfs,cash_flows)
    
    def update(self,new_yields):
        self.yields=new_yields
        self.dfs=discounting(new_yields,time_grid_dcc[1:])


In [ ]:
# generate a new yield curve increment
def yc_scn_gen2(n=3,PCA_mu=PCA_mu,PCA_Lambda=PCA_Lambda,PCA_lambda=PCA_lambda):
    return 22*PCA_mu+np.matmul(PCA_Lambda[:,:n],np.random.multivariate_normal(\
        np.zeros(n),np.diag(22*PCA_lambda[:n])).reshape(n,1)).flatten()

Swaps

In [ ]:

# we concern only vanilla swaps
#  One party pays floating interest, LIBOR 
#  Other party pays the fixed rate, does not worry about movements 
class swap:
    # exchange floating over tenor
    # one party pays floating 
    # other pays fixed rate every month 
    def __init__(self,yc,nominal,maturity,tenor=1): #tenor=1
        self.yc=yc
        self.nominal=nominal #how much money is involved
        self.maturity=maturity # maybe need to be a
        self.montly_cost = yc.cpn(1,self.maturity)
        self.fixed_leg=np.append(np.repeat(self.montly_cost,
            self.maturity),np.repeat(np.zeros(1),N-self.maturity))/12 #*tenor
        # \the fixed cost
        # cash 

    def update(self,steps=1):
        self.maturity-=steps
        # maybe not use, how is it adjusted over roll forward.
        self.fixed_leg=np.append(self.fixed_leg[steps:],np.zeros(steps))


def swap_indicator(maturity):
    # show when do payments due in more general swaps.
    return np.reshape(np.append(np.ones(maturity),np.zeros(N-maturity)),(1,N))

#Our bank will decide to sell og get a swap. 
# Task of deep hedging: Find a swap float to stable

## Objective

In [ ]:
# Possible investements
swap_series=[24,60,120,240]

# BALANCE SHEET PARAMETERS
# assets
A0=100
# liabilities (constant deposits)
L0=50
# control the total swap volume
penalty=0.1

# OPTIMISATION PARAMETERS
target_month=120
# loss function
target_equity=70
# size of training and validation set
nScenarios=10**3
nValidation=10**3
# The four maturities we can go into at any time


## Do nothing bank (benchmark):

In [ ]:
# Over the next 10 year i do nothing 
# Assets will be componded with the montly rate 
# Liabilities will be constant 

def benchmark(nRoutines=nValidation):
    output=np.zeros(nRoutines)
    for l in range(nRoutines):
        # yield curve
        Y=yc(hist_yc(index_yc))
        # assets
        A_pre=A0
        A_post=A0
        L=L0
        E=(A0-L0)
        for k in range(target_month):
            # market updates
            A_pre*=np.exp(Y.yields[0]/12)
            Y=yc(Y.yields+yc_scn_gen())
            A_post=A_pre
            L=L
            E=A_post-L
        output[l]=E
    return output

In [ ]:
#test
Y=yc(hist_yc(index_yc))
np.exp(Y.yields[0]/12)-1, Y.cpn(1,1)/12

## Deep portifolio optimization 

In [ ]:
# netwok parameters
network_params = {"n_hidden_layers": 2, "neurons": 32}
Ktrain = nValidation
Ktest = nValidation


In [ ]:
class SwapDecisionCell(Layer):
    def __init__(self, swap_series, n_hidden_layers, neurons, months, **kwargs):
        self.months = months
        self.max_swap = max(swap_series)
        self.swap_series = swap_series

        self.input_dimension = (
            1  # includes equity , + len(swap_series) with swap prices
        )

        # Number of output dimensions m
        # strategy are all the possible strategies exept 1 moth wich is the rest
        self.output_dimension = len(swap_series)
        # Number of neurons per layer
        self.neurons = neurons
        # Number of hidden layers
        self.n_hidden_layers = n_hidden_layers
        # Activation function
        self.activation = "tanh"  # might shange this later

        self.input_layer = Dense(
            self.neurons,
            activation=self.activation,
            trainable=True,
            kernel_initializer=initializers.RandomNormal(
                0, 1
            ),  # kernel_initializer='random_normal',
            bias_initializer="random_normal",
            name="input",
        )
        self.hidden_layers = [
            Dense(
                neurons,
                activation=self.activation,
                trainable=True,
                kernel_initializer=initializers.RandomNormal(
                    0, 1
                ),  # kernel_initializer='random_normal',
                bias_initializer="random_normal",
                name=f"hidden_{i}",
            )
            for i in range(n_hidden_layers)
        ]

        self.output_layer = Dense(
            self.output_dimension,
            activation="linear",
            trainable=True,
            kernel_initializer=initializers.RandomNormal(
                0, 0.1
            ),  # kernel_initializer='random_normal',
            bias_initializer="random_normal",
            name="output",
        )
        # states are: equity, month, fixed_cash_flows and swap_volume
        self.state_size = [
            1,
            tf.TensorShape(
                [self.max_swap]
            ),  # + max(swap_series) is a hack so that we allways can buy the swaps we want
            tf.TensorShape([self.max_swap]),
        ]

        # For now equity and swap wolume for each timestep
        self.output_size = [1, 1]

        super(SwapDecisionCell, self).__init__(**kwargs)

    def call(self, inputs, states):
        # inputs should be in [(batch, input_1), (batch, input_2, input_3)]
        # state should be in shape [(batch, unit_1), (batch, unit_2, unit_3)]

        ############
        #  Decicion step: time = t
        ############
        equity, fixed_cash_flows, swap_volume = states
        # there is probably some better word than current rate
        # TODO: incorporate prices here? with Concatenate
        swap_rates, current_rate = inputs
        strategy = self.input_layer(swap_rates)
        for layer in self.hidden_layers:
            strategy = layer(strategy)
        strategy = self.output_layer(strategy)

        # Update with our strategy
        for s, swap_length in enumerate(swap_series):
            zeros = (
                tf.tile(strategy[:, s : s + 1], [1, self.max_swap - swap_length]) * 0
            )
            swaps = tf.tile(strategy[:, s : s + 1], [1, swap_length])
            new_swap_volume_s = tf.concat([swaps, zeros], 1)

            swap_volume += new_swap_volume_s
            fixed_cash_flows += new_swap_volume_s * swap_rates[:, s : s + 1]

        #############
        #  Result step:  time = t + 1
        #############

        # the money in the bank gets interst and has to pay interest on the swaps
        equity += (
            (equity - swap_volume[:, 0:1]) * tf.math.exp(current_rate / 12)
            - equity
            - swap_volume[:, 0:1]
        )

        # Loses all equity that it used over swap_volume, i know very strict
        equity -= tf.maximum(0.0, equity * 0.9 - swap_volume[:, 0:1])

        # the bank gets cash from the swaps
        equity += fixed_cash_flows[:, 0:1]
        output = (equity, swap_volume[:, 0:1])

        # update the swap to reflect time has passed
        swap_volume = tf.concat([swap_volume[:, 1:], 0 * swap_volume[:, 0:1]], 1)
        fixed_cash_flows = tf.concat(
            [fixed_cash_flows[:, 1:], 0 * fixed_cash_flows[:, 0:1]], 1
        )

        new_states = (equity, fixed_cash_flows, swap_volume)

        return output, new_states


In [ ]:
# Define network
longest_swap = max(swap_series)
inital_equtiy = Input((1,))
inital_fixed_cash_flows = Input((longest_swap,))
intial_swap_volume = Input((longest_swap,))

initial_states = [inital_equtiy, inital_fixed_cash_flows, intial_swap_volume]

cell = SwapDecisionCell(swap_series=swap_series, months=target_month, **network_params)
rnn_bank_model = RNN(cell)
input_swap_rates = Input(shape=(None, 4))
input_current_rate = Input(shape=(None, 1))
outputs = rnn_bank_model((input_swap_rates, input_current_rate))
bank_model = Model([input_swap_rates, input_current_rate], outputs)


In [ ]:
def get_swap_rate_array(swap_series, sets):
    months = max(swap_series)
    yealds_start = hist_yc(index_yc)
    swap_rate_array = np.zeros((sets, months, len(swap_series)))
    for k in range(sets):
        current_yealds = yealds_start
        for m in range(months):
            for s, swap_length in enumerate(swap_series):
                # Not completly sure if this is calculated correctly
                swap_rate_array[k][m][s] = yc(current_yealds).cpn(1, swap_length) / 12
            current_yealds += yc_scn_gen()
    return swap_rate_array


def get_current_rate_array(months, sets):
    yealds_start = hist_yc(index_yc)
    current_rate_array = np.zeros((sets, months, 1))
    for k in range(sets):
        current_yealds = yealds_start
        for m in range(months):
            # Not completly sure if this is calculated correctly
            current_rate_array[k][m] = current_yealds[0]
            current_yealds += yc_scn_gen()
    return swap_rate_array


In [ ]:
Ktrain = 100
# training data

# inital [equity, fixed_cash_flows, _swap_volume]
inital_state = [
    [(A0 - L0) * np.ones((Ktrain, 1))],
    [np.zeros((Ktrain, longest_swap))],
    [np.zeros((Ktrain, longest_swap))],
]
# swap rates at each time
inputs = [
    get_swap_rate_array(swap_series, Ktrain),
    get_yealds_array(longest_swap, Ktrain),
]

xtrain = inputs  # + inital_state
ytrain = [np.zeros((Ktrain, 1)), np.zeros((Ktrain, 1))]


In [ ]:
# utility function
def u(x):
    pass

# loss function      
def custom_loss(y_true,y_pred):
    return K.mean((y_pred[0]-target_equity)**2 )

In [ ]:
adam=optimizers.Adam(lr=0.01)
bank_model.compile(optimizer=adam,loss=custom_loss)

In [ ]:
#bank_model.summary()
inital_equtiy

In [ ]:
for i in range(1):
    bank_model.fit(x=xtrain,y=ytrain, epochs=20,verbose=True,batch_size=100)

In [ ]:
# generate xtest is not done yet


In [ ]:
equities_nn = bank_model.predict(xtrain)
equities_nn[0].shape

In [ ]:
plt.hist(equities_nn,bins=100,density=True,edgecolor='black',alpha=0.7)
plt.title('Empirical Probability Density Function NN')
plt.xlabel('Equity')
plt.show()

In [ ]:
plt.hist(benchmark(),bins=100,density=True,edgecolor='black',alpha=0.7)
plt.title('Empirical Probability Density Function benchmark')
plt.xlabel('Equity')
plt.show()